# Wrangling OpenStreetMap Data

### Map area: Jakarta, Indonesia

** Data source: https://s3.amazonaws.com/metro-extracts.mapzen.com/jakarta_indonesia.osm.bz2**

---

## Overview

After downloading the map data of Jakarta, I do some initial checking on the document manually.
I then run some code to get the feel of the data.

In [1]:
from osm_datadescriptor import OSMDataDescriptor 

osm_data = OSMDataDescriptor('jakarta_indonesia.osm')

In [2]:
# Basic element check
osm_data.count_element()

{'bounds': 1,
 'member': 2083,
 'nd': 2522657,
 'node': 1994318,
 'osm': 1,
 'relation': 420,
 'tag': 700156,
 'way': 364030}

OSM allows a very flexible tagging system, which gives user freedom but causing problem with consistency.
I count the number of for all tag in the document.

In [3]:
# Check the tag key and element
tag_keys = osm_data.get_tag_keys()

Below I list the top 20 tag keys. As the whole list will be too exhaustive to read.

In [4]:
sorted(tag_keys, key=lambda x: x[1], reverse=True)[:20]

[('building', 557134),
 ('highway', 124402),
 ('created_by', 84423),
 ('name', 79005),
 ('source', 29848),
 ('building:levels', 25345),
 ('building:roof', 25059),
 ('building:walls', 25051),
 ('amenity', 24781),
 ('building:structure', 24658),
 ('flood_prone', 21051),
 ('natural', 19025),
 ('admin_level', 17723),
 ('boundary', 15606),
 ('building:use', 15113),
 ('oneway', 12414),
 ('landuse', 12296),
 ('addr:full', 10038),
 ('kab_name', 10015),
 ('kec_name', 10004)]

Looking through the data I see several things.

There are only a handful usage of postal code total of 900. This is not suprising, since in Indonesia we don't use this postal code that much. 

The other things is the use of phone. This is a bit suprising for me, there are only a handful entry for this (total of 212.
 
And then tag with different case of slightly different spelling presents. 

In [5]:
import re

# Name vs Nama
[item for item in tag_keys if re.match('(name|nama)$', item[0], re.I)]

[('NAMA', 2), ('Nama', 92), ('Name', 2), ('nama', 28), ('name', 79005)]

In [6]:
# Province vs propinsi
[item for item in tag_keys if re.match('(province|propinsi)$', item[0], re.I)]

[('Propinsi', 2), ('Province', 2)]

In [7]:
# Alamat vs address
sorted([item for item in tag_keys if re.match('(addr|alamat)', item[0], re.I)], key=lambda x: x[1], reverse=True)

[('addr:full', 10038),
 ('addr:housenumber', 1475),
 ('addr:street', 1441),
 ('addr:city', 1057),
 ('addr:postcode', 858),
 ('addr:housename', 492),
 ('addr:country', 52),
 ('addr:province', 30),
 ('addr:district', 26),
 ('addr:suburb', 22),
 ('addr:subdistrict', 20),
 ('addr:interpolation', 16),
 ('addr:floor', 4),
 ('addr:state', 4),
 ('ALAMAT', 2),
 ('addr:county', 2),
 ('addr:housenumber_1', 2),
 ('addr:place', 2),
 ('alamat', 2)]

We see for address there are three related tags: 'ALAMAT, 'addr:street', 'addr:full'.
Both 'addr:street' and 'addr:full' is valid tag, so we can not merge them.
[OSM wiki](http://wiki.openstreetmap.org/wiki/Key:addr>) imply that the using 'addr:street' and other supporting field is better then using 'addr:full' but our data shows that we have more 'addr:full' then 'addr:street' (10038 vs 1441).

Indonesia has a bit complex administrative subdivision. It is divided as follow:

* Province
* Regency (Kabupaten) or City (Kota)
* District (Kecamatan)
* Village

For Jakarta (a province), it is divided into 4 cities: Jakarta Selatan, Jakarta Utara, Jakarta Barat and Jakarta Timur.

And then there are non-administrative division like RT and RW. While RT and RW is considered a non-administrative subdivision, it is widely use (The ID card has and requires this information). And from data point of view it is actually interesting to know. A RT have a maximum 30 household.

The division here does not quite match the OSM address tag found here. Which allows for multiple interpretation. And given the complexity of division, not many people can fill this information easily.

I believe this is what lead user to just simply put the whole address in 'addr:full', as this is much simpler. But as OSM warns, putting everything in 'addr:full' makes it harder to parse by software.

There are some effort by the community to try to add the division into the data, but the result is not all that good. For instance for Regency we have 'KAB_NAME', 'kab_name', 'Kabupaten', 'kab.', etc. And some way node uses 'admin_level' tag and then put 'kabupaten' in the value.

Okay let's import the data to MongoDB.

In [8]:
from osm_dataimporter import OSMDataImporter

importer = OSMDataImporter(db_name='osm_data_import', db_collection_name='jakarta')
importer.import_data('jakarta_indonesia.osm')

---

## Problems encountered in Map

### Address prefixes problem

The problems with the address prefixes:

* Abbreviated street names (Jl. Masjid Almunawarah, Jln Perintis, etc)
* Abbreviated alley names (Gg. Kembang)

For the street name there are several variation, that is: 'jl.', 'jln.', 'jl', jln'.
And then some use all upper case, some all lower case and some are mixed.

In [9]:
# Clean up street name
importer.cleanup_address_street()
# Clean up street alley
importer.cleanup_address_alley()

### Address spelling problem

Several street names is actually the same, but spell out differently.
Wikipedia has the name of most of important street in Jakarta. So I will use that as a reference.

In [10]:
import json
import itertools

# Load wikipedia street name data
street_names_file = open('street_name.json')
street_names = json.load(street_names_file)

# Get the street names
flatten_street_names = list(itertools.chain(*street_names.values()))

In [11]:
from pymongo import MongoClient

client = MongoClient()
db = client['osm_data_import']

To check the similarity between the reference address name with the one we have in our MongoDB, I utilize Python built-in [difflib](https://docs.python.org/2/library/difflib.html) library. SequenceMatcher class can compare and give out ratio of similarity. We are intrested at strings that generate ratio above 0.65 and less then 1. We also ignore the prefix and the suffix like 'raya' which is common suffix for street names in Indonesia.

In [12]:
from difflib import SequenceMatcher

# Audit street names
closely_matched_names = set()
for item in db.jakarta.find({'type':'way', 'address.street':{'$exists': True}}):
    for street_name in flatten_street_names:
        # We ignore the street prefix as this will effect the matching ratio.
        shorten_street_name = street_name[6:].lower()
        if shorten_street_name.endswith('raya'):
            shorten_street_name = shorten_street_name[:shorten_street_name.index('raya')].strip()
        item_name = item['address']['street'].lower()
        if item_name.startswith('jalan'):
            item_name = item_name[6:]
        if item_name.endswith('raya'):
            item_name = item_name[:item_name.index('raya')].strip()
            
        s = SequenceMatcher(lambda x: x == " ", item_name, shorten_street_name)
        if 0.65 <= s.ratio() < 1.0:
            closely_matched_names.add((street_name, item['address']['street'], s.ratio(), item['_id']))

closely_matched = sorted(list(closely_matched_names), key=lambda x: x[2], reverse=True)
print "Total similiar item found: ", len(closely_matched)

# Display top 15 with score
[(reference, found, score) for reference, found, score, obj_id in closely_matched[:15]]

Total similiar item found:  140


[(u'Jalan Jenderal Ahmad Yani',
  u'Jalan Jendral Ahmad Yani',
  0.972972972972973),
 (u'Jalan Jenderal Sudirman', u'Jalan Jendral Sudirman', 0.9696969696969697),
 (u'Jalan HR Rasuna Said', u'Jalan HR. Rasuna Said', 0.9655172413793104),
 (u'Jalan TB Simatupang', u'TB. Simatupang', 0.9629629629629629),
 (u'Jalan TB Simatupang', u'TB. Simatupang', 0.9629629629629629),
 (u'Jalan TB Simatupang', u'TB. Simatupang', 0.9629629629629629),
 (u'Jalan Mampang Prapatan', u'Mampang Prapatan 4', 0.9411764705882353),
 (u'Jalan Patimura', u'Pattimura', 0.9411764705882353),
 (u'Jalan HR Rasuna Said', u'Jalan H.R. Rasuna Said', 0.9333333333333333),
 (u'Jalan Gunung Sahari', u'Gunung Sahari 4', 0.9285714285714286),
 (u'Jalan Kapten Tendean', u'Kapten P. Tendean', 0.9032258064516129),
 (u'Jalan HR Rasuna Said', u'Jalan H.R.Rasuna Said', 0.896551724137931),
 (u'Jalan Jenderal Gatot Subroto',
  u'Jalan Jend. Gatot Subroto',
  0.8780487804878049),
 (u'Jalan Veteran', u'Veteran 3', 0.875),
 (u'Jalan MH Thamri

We still need to manually check and replace the street name, but it is a much simpler task.
After manually looking at the result, I found several item that can actually be updated in the database.

In [13]:
from bson.objectid import ObjectId

# Clean up similiar street names
similiar_names = [
  (u'Jalan Jenderal Ahmad Yani', u'Jalan Jendral Ahmad Yani'),
  (u'Jalan Jenderal Sudirman', u'Jalan Jendral Sudirman'),
  (u'Jalan HR Rasuna Said', u'Jalan HR. Rasuna Said'),
  (u'Jalan TB Simatupang', u'TB. Simatupang'),
  (u'Jalan Patimura', u'Pattimura'),
  (u'Jalan HR Rasuna Said', u'Jalan H.R. Rasuna Said'),
  (u'Jalan Kapten Tendean', u'Kapten P. Tendean'),
  (u'Jalan HR Rasuna Said', u'Jalan H.R.Rasuna Said'),
  (u'Jalan Jenderal Gatot Subroto', u'Jalan Jend. Gatot Subroto'),
  (u'Jalan MH Thamrin', u'Jalan M. H. Thamrin'),
  (u'Jalan Abdul Syafiie', u"KH Abdul Syafi'ie"),
  (u'Jalan Jenderal Sudirman', u'Jalan Jend. Sudirman'),
  (u'Jalan Pasar Minggu', u'Jalan Raya Pasar Minggu'),
  (u'Jalan Kyai Haji Wahid Hasyim', u'KH Wahid Hasyim'),
  (u'Jalan Kapten Tendean', u'Kapten Pierre Tendean'),
  (u'Jalan Letnan Jenderal S Parman', u'Letjen S Parman'),
  (u'Jalan Kyai Haji Mas Mansyur', u'K.H. Mas Mansyur'),
  (u'Jalan Letnan Jenderal MT Haryono', u'Jalan Letjen M. T. Haryono'),
  (u'Jalan Jenderal Ahmad Yani', u'Jalan Ahmad Yani'),
  (u'Jalan Laksamana Yos Sudarso', u'Jalan Yos Sudarso'),
  (u'Jalan Insinyur Haji Juanda', u'Jalan Ir. H. Juanda'),
  (u'Jalan Letnan Jenderal S Parman', u'Letjend. Siswondo Parman'),
  (u'Jalan Jenderal Gatot Subroto', u'Gatot Soebroto'),
]

# Update address street
for reference, found in similiar_names:
    res = db.jakarta.update_many({'address.street': found},
                                 {'$set': {'address.street': reference}})
    print "Updated for {} :".format(found), res.modified_count

Updated for Jalan Jendral Ahmad Yani : 1
Updated for Jalan Jendral Sudirman : 1
Updated for Jalan HR. Rasuna Said : 1
Updated for TB. Simatupang : 3
Updated for Pattimura : 1
Updated for Jalan H.R. Rasuna Said : 1
Updated for Kapten P. Tendean : 1
Updated for Jalan H.R.Rasuna Said : 1
Updated for Jalan Jend. Gatot Subroto : 1
Updated for Jalan M. H. Thamrin : 1
Updated for KH Abdul Syafi'ie : 4
Updated for Jalan Jend. Sudirman : 2
Updated for Jalan Raya Pasar Minggu : 1
Updated for KH Wahid Hasyim : 10
Updated for Kapten Pierre Tendean : 4
Updated for Letjen S Parman : 3
Updated for K.H. Mas Mansyur : 1
Updated for Jalan Letjen M. T. Haryono : 4
Updated for Jalan Ahmad Yani : 3
Updated for Jalan Yos Sudarso : 1
Updated for Jalan Ir. H. Juanda : 6
Updated for Letjend. Siswondo Parman : 1
Updated for Gatot Soebroto : 1


In [14]:
# Fix address street that has no prefix
no_prefix_address = [
    (u'Jalan Gunung Sahari', u'Gunung Sahari 4'),
    (u'Jalan Veteran', u'Veteran 3'),
    (u'Jalan Mampang Prapatan', u'Mampang Prapatan 4')
]

for reference, found in no_prefix_address:
    res = db.jakarta.update_many({'address.street': found},
                                 {'$set': {'address.street': 'Jalan {}'.format(found)}}) 
    print "Updated for {}:".format(found), res.modified_count

Updated for Gunung Sahari 4: 1
Updated for Veteran 3: 1
Updated for Mampang Prapatan 4: 1


In [15]:
# Fix address that contain house number
with_housenumber = (u'Jalan Kyai Haji Mas Mansyur', u'KH Mas Mansyur no 57')

db.jakarta.update({'address.street': with_housenumber[1] }, {'$set': {'address.street': with_housenumber[0], 'address.housenumber': 57}})

{u'n': 1, u'nModified': 1, u'ok': 1, 'updatedExisting': True}

### Incomplete city problem

Now that we have normalize the street name, we can actually fill in the city for those addresses.
However if we inspect the data closely, we see that the other area surrounding Jakarta is also included in the data set. For instance Tangerang, Bekasi and Bogor are included. Since it is common in Indonesia to use the same street name for multiple city we need to be conservative here. I will only update the city if there is already a city field and it only says starts with Jakarta. This is really only useful where the city name is uncomplete.

In [16]:
reverse_dict = {}
for k, v in street_names.items():
    for item in v:
        reverse_dict[item] = k

# Audit number of address that have a city 

# Add city whenever possible
for address, city in reverse_dict.items():
    res = db.jakarta.update_many({'address.street': address, 'address.city': re.compile('^jakarta', re.IGNORECASE)},
                                 {'$set': {'address.city': city}})
    if res.modified_count:
        print "Updated for {} :".format(address), res.modified_count

Updated for Jalan Kyai Haji Wahid Hasyim : 8
Updated for Jalan Jenderal Gatot Subroto : 1
Updated for Jalan HR Rasuna Said : 1


### Overuse of addr:full tag

Unfortunately most of the the address is in the addr:full tag instead of addr:street (4515 vs 461). 
If we run:

```python
address_full = list(db.jakarta.find({'address.full':{'$exists': True}}, {'address.full': 1, '_id':0 }))
```

And inspect the output there are a lot of variations. 

- RT RW only no street name, example:

    * RT 002 RW 08
    * RT 0014 RW 010

- Street name with RT no RW, example:
    
    * Jalan sumur batu raya, Rt 07/01
    
- Street name with house number, example:
    
    * Jalan Medan Merdeka Barat No. 12
    * Jalan Kartini 8 dalam no 18


---

## Data Overview

This section contains basic statistics about the dataset and the MongoDB queries used to gather them.
                                                
File sizes:
                                                
jakarta_indonesia.osm ......... 449.2 MB


In [17]:
from pymongo import MongoClient

client = MongoClient()
db = client['osm_data_import']

In [18]:
# Number of document
db.jakarta.find().count()

2358348

In [19]:
# Number of nodes
db.jakarta.find({'type': 'node'}).count()

1994318

In [20]:
# Number of way
db.jakarta.find({'type': 'way'}).count()

363954

In [21]:
# Number of unique user
len(db.jakarta.distinct('created.user'))

1365

In [22]:
# Top 10 contributing user
list(db.jakarta.aggregate([{'$group': {'_id': '$created.user', 'count': {'$sum': 1}}}, {'$sort':{'count':-1}}, {'$limit':10}]))

[{u'_id': u'Alex Rollin', u'count': 409359},
 {u'_id': u'PutriRachiemnys', u'count': 171520},
 {u'_id': u'zahrabanu', u'count': 124793},
 {u'_id': u'Dosandriani', u'count': 114818},
 {u'_id': u'miftajnh', u'count': 114544},
 {u'_id': u'dfo', u'count': 110296},
 {u'_id': u'naomiangelia', u'count': 104560},
 {u'_id': u'Firman Hadi', u'count': 96807},
 {u'_id': u'anisa berliana', u'count': 89299},
 {u'_id': u'ceyockey', u'count': 70948}]

In [23]:
# Place of worship breakdown
list(db.jakarta.aggregate([
        {"$match":{"amenity":{"$exists":1}, "amenity":"place_of_worship"}},
        {"$group":{"_id":"$religion", "count":{"$sum":1}}},
        {"$sort":{"count":-1}}
    ]))

[{u'_id': u'muslim', u'count': 3438},
 {u'_id': u'christian', u'count': 374},
 {u'_id': u'buddhist', u'count': 68},
 {u'_id': None, u'count': 68},
 {u'_id': u'hindu', u'count': 15},
 {u'_id': u'confucian', u'count': 4}]

---

## Additional ideas

Jakarta experience flooding issue every year. There is this cycle the citizen believe, small flood every year and a big one every 5 years.

Looking at the tag list, I saw this:

    ('flood:overflow', 2619),
    ('flood:rain', 4859),
    ('flood:rob', 1049),
    ('flood:send', 3362),
    ('flood_cause:overflowing_river', 2),
    ('flood_depth', 5860),
    ('flood_duration', 5696),
    ('flood_latest', 5845),
    ('flood_prone', 21051),
    ('floodprone', 19)

Which is great, so we have flooding information.
But, I imagine it will be difficult to manually add this information.

Fortunately, Indonesian loves Twitter, and they tweet about the event everytime this happens.
Some of the user turn on their geolocation. So we can probably use that to populate more flooding information into our data. Use Twitter API to fetch user flood information get the geolocation, (if needed use Google API to do geo reverse and add entry to OSM data) and update the data.

---

## Conclusion and Notes

The data we obtain from OSM is far from perfect. For the purpose of this exercise, however, I have clean up the address.

I wish I could clean the full address ('address.full' key) a bit, but there is no format, which makes it really painful to parse. 

Capitalization is also a problem in the data set. But I can not find the list of address to be the reference. We can simply perform capitilization on the address but I don't think this is accurate. For instance one of the address is 'kh mas mansyur' which we can not immediately capitalize to 'Kh Mas Mansyur' as the correct capitalization is 'KH Mas Mansyur'.

The data set also includes surrounding city like Tangerang, Bekasi, Bogor, etc. So a better name for the dataset will be [Greater Jakarta](https://en.wikipedia.org/wiki/Greater_Jakarta). 